In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Softsign activation (used in the paper)
class Softsign(nn.Module):
    def forward(self, x):
        return x / (1 + torch.abs(x))


In [4]:
def glorot_init(tensor):
    if tensor.dim() < 2:
        raise ValueError("Tensor must have at least 2 dimensions for Glorot init.")
    
    fan_in, fan_out = tensor.size(1), tensor.size(0)
    bound = math.sqrt(6.0 / (fan_in + fan_out))
    with torch.no_grad():
        tensor.uniform_(-bound, bound)
        
linear = nn.Linear(256, 512)
glorot_init(linear.weight)
nn.init.zeros_(linear.bias)


Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [7]:
class MLP(nn.Module):
    def __init__(self, layer_sizes, activation="tanh"):
        super().__init__()
        
        acts = {
            "sigmoid": nn.Sigmoid(),
            "tanh": nn.Tanh(),
            "softsign": Softsign(),
        }
        self.activation = acts[activation]
        
        layers = []
        for i in range(len(layer_sizes)-1):
            in_f, out_f = layer_sizes[i], layer_sizes[i+1]
            linear = nn.Linear(in_f, out_f)
            glorot_init(linear.weight)     # ← our Xavier initialization
            nn.init.zeros_(linear.bias)
            layers.append(linear)
        
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for linear in self.layers[:-1]:
            x = self.activation(linear(x))
        return self.layers[-1](x)  # last layer linear

model = MLP([784, 1000, 1000, 10], activation="softsign")

print(model)

MLP(
  (activation): Softsign()
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=1000, bias=True)
    (2): Linear(in_features=1000, out_features=10, bias=True)
  )
)


In [8]:
def check_activation_statistics(model, input_dim=784, samples=1000):
    x = torch.randn(samples, input_dim)
    
    activations = []
    with torch.no_grad():
        for layer in model.layers[:-1]:
            x = model.activation(layer(x))
            activations.append(x.clone())

    for i, a in enumerate(activations):
        print(f"Layer {i+1}: mean={a.mean():.4f}, std={a.std():.4f}")


In [9]:
check_activation_statistics(MLP([784, 1000, 1000, 1000], "tanh"))


Layer 1: mean=0.0007, std=0.6088
Layer 2: mean=-0.0001, std=0.4761


In [10]:
def check_gradient_flow(model, input_dim=784, batch=32):
    x = torch.randn(batch, input_dim)
    y = torch.randint(0, 10, (batch,))
    criterion = nn.CrossEntropyLoss()

    out = model(x)
    loss = criterion(out, y)
    loss.backward()

    for i, layer in enumerate(model.layers):
        grad = layer.weight.grad
        print(f"Layer {i+1} grad std = {grad.std():.6f}")


In [11]:
check_gradient_flow(MLP([784, 512, 512, 10], "softsign"))


Layer 1 grad std = 0.003128
Layer 2 grad std = 0.003035
Layer 3 grad std = 0.014662


In [12]:
def estimate_jacobian_singular_values(layer, x):
    x = x.requires_grad_(True)
    y = layer(x)
    
    jacobian = []
    for i in range(y.shape[1]):  
        grad_outputs = torch.zeros_like(y)
        grad_outputs[:, i] = 1.0
        grad = torch.autograd.grad(y, x, grad_outputs=grad_outputs, retain_graph=True)[0]
        jacobian.append(grad)
    
    J = torch.stack(jacobian, dim=1)  # shape: [batch, out, in]
    J = J.mean(0)                      # average over batch

    # compute singular values
    U, S, V = torch.svd(J)
    return S


In [13]:
layer = nn.Linear(100, 100)
glorot_init(layer.weight)

x = torch.randn(32, 100)
sv = estimate_jacobian_singular_values(layer, x)

print("Average singular value:", sv.mean().item())


Average singular value: 0.849372386932373


In [14]:
def train_demo(activation="tanh"):
    model = MLP([784, 1000, 1000, 10], activation=activation)
    opt = torch.optim.SGD(model.parameters(), lr=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for step in range(200):
        x = torch.randn(64, 784)
        y = torch.randint(0, 10, (64,))

        out = model(x)
        loss = loss_fn(out, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        if step % 20 == 0:
            print(f"[{activation}] Step {step}, Loss = {loss.item():.4f}")


In [15]:
train_demo("sigmoid")
train_demo("tanh")
train_demo("softsign")


[sigmoid] Step 0, Loss = 2.6037
[sigmoid] Step 20, Loss = 2.3526
[sigmoid] Step 40, Loss = 2.4421
[sigmoid] Step 60, Loss = 2.3401
[sigmoid] Step 80, Loss = 2.4809
[sigmoid] Step 100, Loss = 2.3195
[sigmoid] Step 120, Loss = 2.3395
[sigmoid] Step 140, Loss = 2.3540
[sigmoid] Step 160, Loss = 2.4798
[sigmoid] Step 180, Loss = 2.3993
[tanh] Step 0, Loss = 2.4091
[tanh] Step 20, Loss = 2.3892
[tanh] Step 40, Loss = 2.5889
[tanh] Step 60, Loss = 2.6190
[tanh] Step 80, Loss = 2.4979
[tanh] Step 100, Loss = 2.6531
[tanh] Step 120, Loss = 2.3152
[tanh] Step 140, Loss = 2.4932
[tanh] Step 160, Loss = 2.4560
[tanh] Step 180, Loss = 2.4853
[softsign] Step 0, Loss = 2.2867
[softsign] Step 20, Loss = 2.2622
[softsign] Step 40, Loss = 2.4207
[softsign] Step 60, Loss = 2.3916
[softsign] Step 80, Loss = 2.3379
[softsign] Step 100, Loss = 2.3843
[softsign] Step 120, Loss = 2.4374
[softsign] Step 140, Loss = 2.3562
[softsign] Step 160, Loss = 2.3907
[softsign] Step 180, Loss = 2.3665
